<p float="left">
  <img src="images/arrow_logo.jpg" alt="Arrow Logo" width="200" height="100" align="left"/>
  <img src="images/adi_logo.png" alt="ADI Logo" width="200" height="100" align="right"/>
</p>

<hr style="height:5px; text-align:bottom">

## Modifying the AD7124 On-Chip Registers

1.Restart the kernel and clear the output by clicking <strong>Kernel -> Restart & Clear Output</strong> in the toolbar at the top of the page. This clears the kernel of any variables or functions that may have been created in the previous program.

2.Create a copy of the ad7124_default_config.py file and name it 'ad7124_custom_config.py'. This newly created file will be used for all of the custom modification you would like to make.

In [ ]:
import shutil

shutil.copy2('ad7124_default_config.py', 'ad7124_custom_config.py')

3.Modify the default values for the AD7124 registers as needed, following the  [AD7124 datasheet](https://www.analog.com/media/en/technical-documentation/data-sheets/ad7124-8.pdf) for reference

##### A. First we need to import the Serial and Data modules required for the demo. We will also set the Matplotlib backend to provide a pop-up window for our data plots

In [ ]:
from safari_serial import Serial
from safari_data import Data

# Set the TkAgg backend for matplotlib
%matplotlib tk

4.Next we need to update the config file, this is done by simply passing the name of the custom configuration file to the Serial class when it is initialized. This is done for you in the cell below.

<span style='color:red'><strong>You should modify the 'channels' data structure according to your AD7124 configuration and application needs. The 'channels' structure used below is for reference only</strong></span>

To edit the cell click inside it. The border will turn green.

In [ ]:
from safari_serial import Serial
from safari_data import Data

# Set the TkAgg backend for matplotlib
%matplotlib tk

# Add entries for each enabled channel. For example if you only enabled channel 5 in your custom configuration:
channels = {"0":{}}

#default fields for each channel
for channel in channels:
    channels[channel]["max_elements"] = 300
    channels[channel]["timestamps"]=[]
    channels[channel]["voltages"]=[]
    channels[channel]["values"]=[]
    channels[channel]["axis"] = None
    channels[channel]["annotation"] = None
    channels[channel]["annotation_visible"] = False
    channels[channel]["annotation_text"] = ""
    channels[channel]["line"] = None
    channels[channel]["annot_ax_xy"] = None
    channels[channel]["axes_color"] = 'k'
    channels[channel]["plot_title"] = f"Channel {channel}"
    channels[channel]["y_label"] = ""
    channels[channel]["y_label_converted"] = ""
    channels[channel]["voltage_format_func"] = None
    channels[channel]["value_conversion_func"] = None
    
# Function for converting channel 0 voltage to an accelerometer value
def convert_accelerometer(voltage, code):
    del code
    
    vcc = 3.3
    return round((voltage - vcc / 2) * 1 / 0.640, 3)
    
# Sample Configuration Channel 0 - Configure according to your application
channels["0"]["axes_color"] = 'tab:blue'
channels["0"]["plot_title"] = "Channel 0"
channels["0"]["y_label"] = 'Voltage (V)'
channels["0"]["y_label_converted"] = 'Accel. (g)'
channels["0"]["value_conversion_func"] = convert_accelerometer

serial = Serial(channels, 'ad7124_custom_config.py')
data = Data(channels)

5.It is good practice to update the AD7124 python driver (ad7124.py) as well as the data collection application (safari.py) on the Meerkat as well. Execute the below cells <strong>AFTER</strong> successfully connecting the serial port.

In [ ]:
serial.write_file_to_meerkat("ad7124.py", "ad7124.py")

In [ ]:
serial.write_file_to_meerkat("safari.py", "safari.py")